<a href="https://colab.research.google.com/github/nofarmordehai/TIC-TAC-TOE-maxmin/blob/master/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Play TIC TAC TOE Against a smart computer**

1. choose "Runtime" -> "Run all"
2. collapse the code using the arrow (or read it if you're curious!)
3. go to "PLAY!" and try to beat the computer (more instructions there)



# Game & Board code


In [ ]:
class Board:

  # create board

  def __init__ (self):
    
    self.board = {1: ' ' , 2: ' ' , 3: ' ' ,
            4: ' ' , 5: ' ' , 6: ' ' ,
            7: ' ' , 8: ' ' , 9: ' '}
    self.is_winner = False
    self.count_full = 0
    self.player_won = ' '
    self.player_starts = ' '

  # win checks

  def win_row(self):
    row_start_point = [1, 4, 7]
    for start in row_start_point:
      if self.board[start] != ' ' and self.board[start] == self.board[start+1] == self.board[start+2]:
        self.is_winner = True
        self.player_won = self.board[start]
        break
  
  def win_column(self):
    row_start_point = [1, 2, 3]
    for start in row_start_point:
      if self.board[start] != ' ' and self.board[start] == self.board[start+3] == self.board[start+6]:
        self.is_winner = True
        self.player_won = self.board[start]
        break

  def win_diagonal(self):
    if self.board[5] != ' ' and (self.board[1] == self.board[5] == self.board[9] or self.board[3] == self.board[5] == self.board[7]):
      self.is_winner = True
      self.player_won = self.board[5]
    
  def check_win(self):
    self.win_row()
    if self.is_winner == False:
      self.win_column()
      if self.is_winner == False:
        self.win_diagonal()

  # prints & instructions

  def print_board(self):
    print(self.board[1] + '|' + self.board[2] + '|' + self.board[3])
    print('-+-+-')
    print(self.board[4] + '|' + self.board[5] + '|' + self.board[6])
    print('-+-+-')
    print(self.board[7] + '|' + self.board[8] + '|' + self.board[9])

  def print_instructions(self):
    print("HELLO! welcome to TIC-TAC-TOE")
    print("This is our board cells number markings:")
    print()
    print('1|2|3')
    print('-+-+-')
    print('4|5|6')
    print('-+-+-')
    print('7|8|9')
    print()
    print("When asked please play your turn and enter avaliable cell number for X.")
  
  def who_starts(self):
    while self.player_starts != 'X' and self.player_starts != 'O':
      print("You're X I'm O. Who goes first ? please enter X or O.")
      self.player_starts = input()

  def message_end(self):
    print("GAME OVER!")
    if self.is_winner:
      print(self.player_won,"WON!")
    else:
      print("TIE!")


  def avaliable_cells(self):
    avaliables = []
    for i in range(1,10):
      if self.board[i] == ' ': 
        avaliables.append(i)
    return avaliables

  # player_turn

  def player_turn(self):
    turn = 0
    avaliables = self.avaliable_cells()
    while turn not in avaliables:
      print("Your turn! Please enter avaliable cell number for X.")
      turn = input()
      turn = ord(turn) - 48
    self.board[turn] = 'X'
  
  def total_player_turn(self):
      self.player_turn()
      self.count_full = self.count_full + 1 
      self.print_board()
      self.check_win()

   # computer_turn

  def computer_turn(self):
    turn = best_move(self)
    self.board[turn] = 'O'
    #print ("computer still learning :(")

  def total_computer_turn(self):
      self.computer_turn()
      self.count_full = self.count_full + 1 
      print ("Computer turn :)")
      self.print_board()
      self.check_win()
  

In [ ]:
def new_game():
  board = Board()
  board.print_instructions()
  board.who_starts()
  
  print('Lets start!')
  board.print_board()

  if board.player_starts == 'X':
    while board.is_winner == False and board.count_full < 9:
      board.total_player_turn()
      if board.is_winner or board.count_full == 9: break
      board.total_computer_turn()
  
  elif board.player_starts == 'O':
    while board.is_winner == False and board.count_full < 9:
      board.total_computer_turn()
      if board.is_winner or board.count_full == 9: break
      board.total_player_turn()

  board.message_end()

# Computer logic code (maxmin)


In [ ]:
import random

In [ ]:
def do_move(board, move, player):
  board.board[move] = player
  board.count_full = board.count_full + 1

def undo_move(board, move):
  board.board[move] = ' '
  board.count_full = board.count_full - 1    
  board.is_winner = False
  board.player_won = ' '

In [ ]:
def best_move(board):

  # computer is first, corner is best.
  if board.count_full == 0:
    return random.choice([1,3,7,9]) 

  # fill last cell.
  if board.count_full == 8:
    for i in range(1,10):
      if board.board[i] == ' ': 
        return i
  
  # computer is second, player 'X' in a corner, so 'O' in the middle.
  if board.count_full == 1:
    for i in [1,3,7,9]:
      if board.board[i] != ' ':
        return 5 

  # maxmin method
  avaliables = board.avaliable_cells()

  best = -1000
  best_turn = 0

  for move in avaliables:
    do_move(board, move, 'O')
    
    # value move
    check = maxmin(board,0, False)
    if check > best: 
      best = check
      best_turn = move

    undo_move(board, move)

  return best_turn

In [ ]:
def evaluate(board):
  board.check_win()
  if board.is_winner:
    return 10 if board.player_won == 'O' else -10
  if board.count_full == 9: # and no winner, tie
    return 0
  return ' '

In [ ]:
def maxmin(board, depth, isMax):
  
  score = evaluate(board)
  if score != ' ':
    return score

  avaliables = board.avaliable_cells()

  # If this maximizer's move 
  if isMax:
    best = -1000
    for move in avaliables:
      do_move(board, move, 'O')
      check = maxmin(board,depth+1, not isMax)
      best = check if check > best else best
      undo_move(board, move)
    return best-depth

  # If this minimizer's move 
  else:
    best = 1000
    for move in avaliables:
      do_move(board, move, 'X')
      check = maxmin(board, depth+1,not isMax)
      best = check if check < best else best
      undo_move(board, move)
    return best+depth


# PLAY!

In [ ]:
new_game()

HELLO! welcome to TIC-TAC-TOE
This is our board cells number markings:

1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

When asked please play your turn and enter avaliable cell number for X.
You're X I'm O. Who goes first ? please enter X or O.
O
Lets start!
 | | 
-+-+-
 | | 
-+-+-
 | | 
Computer turn :)
 | | 
-+-+-
 | | 
-+-+-
O| | 
Your turn! Please enter avaliable cell number for X.
3
 | |X
-+-+-
 | | 
-+-+-
O| | 
Computer turn :)
O| |X
-+-+-
 | | 
-+-+-
O| | 
Your turn! Please enter avaliable cell number for X.
4
O| |X
-+-+-
X| | 
-+-+-
O| | 
Computer turn :)
O| |X
-+-+-
X| | 
-+-+-
O| |O
Your turn! Please enter avaliable cell number for X.
8
O| |X
-+-+-
X| | 
-+-+-
O|X|O
Computer turn :)
O| |X
-+-+-
X|O| 
-+-+-
O|X|O
GAME OVER!
O WON!
